In [6]:
import pandas as pd

df1 = pd.read_csv("fake_job_postings.csv")
df1.drop(['benefits', 'company_profile', 'employment_type', 'salary_range',
          'industry', 'department', 'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)
df2 = pd.read_csv("job_train.csv")
df2 = df2[df2['fraudulent']==1]
merged_df = pd.concat([df1, df2])
# merged_df.to_csv("final_data.csv")
merged_df

,title,location,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent
0,Marketing Intern,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0
3,Account Executive - Washington DC,"US, DC, Washington",THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,0
4,Bill Review Manager,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0
...,...,...,...,...,...,...,...,...
8883,Military Benefits Counselor,"US, , chicago",FOR MILITARY ONLYIf you can call present and p...,must be:Active duty or veteran servicememberha...,1,1,1,1
8886,Cruise Staff Wanted *URGENT*,"US, NY, NEWYORK CITY",6* Ultra Luxury American Cruise Company is urg...,Certification &amp; Experience: Previous exper...,0,0,1,1
8897,"Account Sales Managers $80-$130,000/yr","US, NY, Rochester",We have aggressive growth plans in place for t...,Experience and Skills Required- Minimum of 3 ...,0,1,0,1
8907,Research Engineer Data Science,"US, CA, San Mateo",Qualified candidates are encouraged to apply d...,Position requirements: Quickly learning new ar...,0,1,1,1


In [7]:
# print(merged_df.isnull().sum())
print(df1.duplicated().sum())
print(df2.duplicated().sum())
print(merged_df.duplicated().sum())

df1_unique = df1.drop_duplicates()
df2_unique = df2.drop_duplicates()
overlap_count = pd.merge(df1_unique, df2_unique, how='inner').shape[0]
print(overlap_count)
print(len(df2))

397
10
853
446
456


In [4]:
merged_df.info()
print(merged_df.nunique())
merged_df.duplicated().sum()
merged_df.drop_duplicates(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 18336 entries, 0 to 8938
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             18336 non-null  object
 1   location          17982 non-null  object
 2   description       18334 non-null  object
 3   requirements      15564 non-null  object
 4   telecommuting     18336 non-null  int64 
 5   has_company_logo  18336 non-null  int64 
 6   has_questions     18336 non-null  int64 
 7   fraudulent        18336 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 1.3+ MB
title               11231
location             3105
description         14801
requirements        11967
telecommuting           2
has_company_logo        2
has_questions           2
fraudulent              2
dtype: int64


In [69]:
textual = ['title', 'company_profile', 'description',
       'requirements', 'benefits']
categorical = ['location', 'employment_type', 'industry']
continuous = ['salary_range']
label = ['fraudulent']

merged_df.isnull().sum()

title                  0
location             354
description            2
requirements        2772
telecommuting          0
has_company_logo       0
has_questions          0
fraudulent             0
dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Combine relevant text columns (you can modify this based on which columns you want to use)
merged_df['text'] = merged_df['title'].fillna('') + ' ' + merged_df['description'].fillna('')

# Split the data
X = merged_df['text']
y = merged_df['fraudulent']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Make predictions and print classification report
y_pred = model.predict(X_test_vec)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(model.coef_)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3321
           1       0.95      0.35      0.51       176

    accuracy                           0.97      3497
   macro avg       0.96      0.67      0.75      3497
weighted avg       0.97      0.97      0.96      3497

[[ 0.28998688  1.26500827 -0.21935576 ... -0.17891268 -0.05137752
  -0.06817463]]


/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


np.int64(0)

In [ ]:

# len(df3)
# len(merged_df)

18336

In [36]:
merged_df.drop_duplicates(inplace=True)
# Combine relevant text columns (you can modify this based on which columns you want to use)
merged_df['text'] = merged_df['title'].fillna('') + ' ' + merged_df['description'].fillna('')

# Split the data
X = merged_df['text']
y = merged_df['fraudulent']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Make predictions and print classification report
y_pred = model.predict(X_test_vec)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(model.coef_)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3321
           1       0.95      0.35      0.51       176

    accuracy                           0.97      3497
   macro avg       0.96      0.67      0.75      3497
weighted avg       0.97      0.97      0.96      3497

[[ 0.28998688  1.26500827 -0.21935576 ... -0.17891268 -0.05137752
  -0.06817463]]


/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/woojungyi/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


In [25]:
merged_df.duplicated().sum()

np.int64(0)

In [ ]:
import pandas as pd

df1 = pd.read_csv("fake_job_postings.csv")
df1.drop(['benefits', 'company_profile', 'employment_type', 'salary_range', 'industry', 'department', 'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)
df2 = pd.read_csv("job_train.csv")
merged_df = pd.concat([df1, df2])

merged_df

In [63]:
##### Original Ver. #####

df1 = pd.read_csv("fake_job_postings.csv")
df1.drop(['department', 'telecommuting', 'has_company_logo', 'has_questions',
        'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)
df2 = pd.read_csv("fake_postings.csv")
merged_df = pd.concat([df1, df2])

In [64]:
# print(merged_df.isnull().sum())
print(df1.duplicated().sum())
print(df2.duplicated().sum())
print(merged_df.duplicated().sum())

df1_unique = df1.drop_duplicates()
df2_unique = df2.drop_duplicates()
overlap_count = pd.merge(df1_unique, df2_unique, how='inner').shape[0]
print(overlap_count)
print(len(df2))

314
0
314
0
10000
